In [ ]:
!pip install google-cloud-vision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import os
from google.oauth2 import service_account
import cv2
import numpy as np
import tensorflow as tf
from datetime import datetime

In [8]:
# Set the path
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/path/to.service_account_key.json'

In [9]:
from requests.api import get
from google.cloud import vision
from google.cloud.vision_v1 import types

def detect_food_labels(image):
  # Loading the input image
  with open(image, 'rb') as f:
    image_content = f.read()
  # Initializing API Client
  credentials = service_account.credentials.from_service_account_file(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])
  client = vision.ImageAnnotatorClient(credentials=credentials)

  # Calling Client for food detection
  image = types.image(content=image)
  resp = client.object_locatization(image=image)
  food_labels = []
  for obj in resp.localized_object_annotations:
    if obj.name.lower() in ['food', 'dish']:
      food_labels.append({
          'name':obj.name,
          'score': obj.score,
          'calories': obj.properties['nutrition'].get('energy', {}).get('amount', None),
          'protein': obj.properties['nutrition'].get('protein', {}).get('amount', None),
          'fat': obj.properties['nutrition'].get('fat', {}).get('amount', None),
                'carbohydrates': obj.properties['nutrition'].get('carbohydrate', {}).get('amount', None)
      })
  return food_labels

In [10]:
def sort_food_by_type_and_expiry(food_objects):
    # Create a dictionary to store the food objects by type
    food_by_type = {}
    for obj in food_objects:
        food_type = obj['type']
        if food_type not in food_by_type:
            food_by_type[food_type] = []
        food_by_type[food_type].append(obj)

    # Sort the food objects within each type by expiry date
    for food_type in food_by_type:
        food_objs = food_by_type[food_type]
        food_objs.sort(key=lambda x: datetime.strptime(x['expiry'], '%m/%d/%Y'))

    # Return the sorted food objects as a list
    sorted_food = []
    for food_type in food_by_type:
        sorted_food += food_by_type[food_type]

    return sorted_food


In [11]:
def detect_and_sort_food_objects(image_path, detect_fn, category_index, min_score_thresh=0.5):
    # Load the image
    image_np = cv2.imread(image_path)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Run object detection
    detections = detect_fn(input_tensor)

    # Filter the detected objects to only include food objects
    food_detection_classes = [44, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89]
    detection_boxes = detections['detection_boxes'][0].numpy()
    detection_classes = detections['detection_classes'][0].numpy().astype(np.uint8)
    detection_scores = detections['detection_scores'][0].numpy()
    food_detection_boxes = []
    food_detection_classes = []
    food_detection_scores = []
    food_detection_labels = []
    food_detection_types = []
    food_detection_expiry = []
    for i in range(len(detection_boxes)):
        if detection_classes[i] in food_detection_classes and detection_scores[i] > min_score_thresh:
            food_detection_boxes.append(detection_boxes[i])
            food_detection_classes.append(detection_classes[i])
            food_detection_scores.append(detection_scores[i])
            food_detection_labels.append(category_index[detection_classes[i]]['name'])
            food_detection_types.append(category_index[detection_classes[i]]['name'].split(':')[0])
            food_detection_expiry.append('01/01/2024')  # Replace with your code to extract the expiry date from the image

    # Combine the detected food objects into a list
    food_objects = []
    for i in range(len(food_detection_boxes)):
        food_objects.append({
            'label': food_detection_labels[i],
            'type': food_detection_types[i],
            'score': food_detection_scores[i],
            'box': food_detection_boxes[i],
            'expiry': food_detection_expiry[i]
        })

    # Sort the detected food objects by type and expiry date
    food_objects_sorted = sorted(food_objects, key=lambda x: (x['type'], datetime.strptime(x['expiry'], '%m/%d/%Y')))

    return food_objects_sorted
    

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def create_food_detection_model():
    # Define the model architecture
    model = tf.keras.Sequential([
        layers.Input(shape=(None, None, 3)),
        layers.Conv2D(32, 3, activation='relu', padding='same'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu', padding='same'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation='relu', padding='same'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])

    # Compile the model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


In [16]:
import cv2
import numpy as np
import tensorflow as tf
from datetime import datetime

def load_yolo_model():
    # Load the YOLOv3 model
    model = tf.keras.models.load_model('yolov3.h5')

    # Build the model input pipeline
    input_size = 416
    max_boxes = 20
    iou_threshold = 0.5
    score_threshold = 0.5
    classes_file = 'coco.names'

    # Load the class names
    with open(classes_file, 'r') as f:
        class_names = f.read().splitlines()

    # Create a dictionary mapping class names to indices
    class_indices = {name: i for i, name in enumerate(class_names)}

    # Create a function to convert YOLOv3 output to bounding boxes
    def yolo_boxes(predictions, anchors, num_classes):
        grid_size = tf.shape(predictions)[1:3]
        strides = (input_size[0] // grid_size[0], input_size[1] // grid_size[1])
        bbox_attrs = 5 + num_classes
        num_anchors = len(anchors)

        # Reshape the predictions into a grid
        predictions = tf.reshape(predictions, [-1, grid_size[0], grid_size[1], num_anchors, bbox_attrs])

        # Compute the box centers, widths, and heights
        box_xy = tf.sigmoid(predictions[..., :2])
        box_wh = tf.exp(predictions[..., 2:4]) * anchors

        # Compute the confidence score and class probabilities
        box_confidence = tf.sigmoid(predictions[..., 4:5])
        box_class_probs = tf.sigmoid(predictions[..., 5:])

        # Compute the grid offsets
        grid_x = tf.range(grid_size[1], dtype=tf.float32)
        grid_y = tf.range(grid_size[0], dtype=tf.float32)
        grid_x, grid_y = tf.meshgrid(grid_x, grid_y)
        x_offset = tf.reshape(grid_x, (-1, 1))
        y_offset = tf.reshape(grid_y, (-1, 1))
        xy_offset = tf.concat([x_offset, y_offset], axis=-1)
        xy_offset = tf.tile(xy_offset, [1, num_anchors])
        xy_offset = tf.reshape(xy_offset, [grid_size[0], grid_size[1], num_anchors, 2])

        # Apply the grid offsets to the box centers
        box_xy += xy_offset

        # Apply the stride to the box centers and dimensions
        box_xy *= strides
        box_wh *= strides

        # Compute the box corners
        box_mins = box_xy - 0.5 * box_wh
        box_maxes = box_xy + 0.5 * box_wh
        boxes = tf.concat([box_mins[..., 1:], box_maxes[..., 1:]], axis=-1)

        return boxes, box_confidence, box_class_probs

    def yolo_nms(outputs, anchors, num_classes, score_threshold=0.5, iou_threshold=0.5, max_boxes=10):
        boxes, conf, class_probs = yolo_boxes(outputs, anchors, num_classes)

        scores = conf * class_probs
        boxes = tf.reshape(boxes, [-1, 4])
        scores = tf.reshape(scores, [-1, num_classes])
        class_indices = tf.argmax(scores, axis=1)
        class_scores = tf.reduce_max(scores, axis=1)

        # Apply score threshold
        score_mask = class_scores >= score_threshold
        boxes = tf.boolean_mask(boxes, score_mask)
        class_indices = tf.boolean_mask(class_indices, score_mask)
        class_scores = tf.boolean_mask(class_scores, score_mask)

        # Apply non-max suppression
        selected_indices = tf.image.non_max_suppression(
            boxes, class_scores, max_output_size=max_boxes, iou_threshold=iou_threshold
        )
        selected_boxes = tf.gather(boxes, selected_indices)
        selected_scores = tf.gather(class_scores, selected_indices)
        selected_class_indices = tf.gather(class_indices, selected_indices)

        return selected_boxes, selected_scores, selected_class_indices

    def yolo_filter_boxes(boxes, scores, score_threshold=0.6, iou_threshold=0.5):
        """
        Filters YOLO boxes based on object and class confidence scores and performs
        non-max suppression to remove overlapping boxes.

        Arguments:
        boxes -- tensor of shape (N, 4) containing the coordinates of N boxes
        scores -- tensor of shape (N,) containing the confidence scores of N boxes
        score_threshold -- float, threshold for removing boxes with a score less than this
        iou_threshold -- float, threshold for removing overlapping boxes

        Returns:
        filtered_boxes -- tensor of shape (M, 4) containing the coordinates of M filtered boxes
        filtered_scores -- tensor of shape (M,) containing the confidence scores of M filtered boxes
        filtered_classes -- tensor of shape (M,) containing the class indices of M filtered boxes
        """

        # Apply score threshold
        indices = tf.where(scores >= score_threshold)
        filtered_boxes = tf.gather_nd(boxes, indices)
        filtered_scores = tf.gather_nd(scores, indices)
        
        # Apply non-max suppression
        selected_indices, selected_scores = tf.image.non_max_suppression_with_scores(filtered_boxes, 
                                                                                      filtered_scores, 
                                                                                      max_output_size=100, 
                                                                                      iou_threshold=iou_threshold, 
                                                                                      score_threshold=score_threshold)
        filtered_boxes = tf.gather(filtered_boxes, selected_indices)
        filtered_scores = selected_scores
        filtered_classes = tf.argmax(scores, axis=1)
        filtered_classes = tf.gather(filtered_classes, selected_indices)
        
        return filtered_boxes, filtered_scores, filtered_classes


    def detect_objects(image_path, model, class_names, anchors, input_size=(416, 416), score_threshold=0.5, iou_threshold=0.5):
        # Load the image using OpenCV
        image = cv2.imread(image_path)

        # Preprocess the image
        image = cv2.resize(image, input_size)
        image = image.astype(np.float32) / 255.0

        # Call the YOLOv3 model on the preprocessed image
        outputs = model.predict(np.expand_dims(image, axis=0))

        # Apply the yolo_boxes, yolo_nms, and yolo_filter_boxes functions to obtain the final detections
        boxes, scores, classes = yolo_nms(outputs, anchors, len(class_names), score_threshold=score_threshold, iou_threshold=iou_threshold)

        # Draw the final boxes on the input image
        for box, cls in zip(boxes, classes):
            x1, y1, x2, y2 = box.astype(np.int32)
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(image, class_names[cls], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display the final image
        cv2.imshow("Detection", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        # Assuming that you have a model object named `model`
        tf.saved_model.save(model, 'saved_model')

    def main():
      image_path = 'input.jpg'
      detect_fn = tf.saved_model.load('saved_model')
      category_index = {
          1: {'id': 1, 'name': 'apple'},
          2: {'id': 2, 'name': 'banana'},
          3: {'id': 3, 'name': 'orange'},
          4: {'id': 4, 'name': 'pear'},
          5: {'id': 5, 'name': 'watermelon'},
          6: {'id': 6, 'name': 'tomato'},
          7: {'id': 7, 'name': 'potato'},
          8: {'id': 8, 'name': 'carrot'},
          9: {'id': 9, 'name': 'broccoli'},
          10: {'id': 10, 'name': 'pizza'},
          11: {'id': 11, 'name': 'cake'},
          12: {'id': 12, 'name': 'donut'},
          13: {'id': 13, 'name': 'chair'}
      }
      min_score_thresh = 0.5

      # Detect and sort the food objects in the input image
      food_objects_sorted = detect_and_sort_food_objects(image_path, detect_fn, category_index, min_score_thresh)

      # Print the sorted food objects to the console
      for obj in food_objects_sorted:
          print(obj)

    if __name__ == '__main__':
          main()